### 1. Import libraries

In [1]:
import gc
import json
import os
import math
import multiprocessing
import numpy as np
import pandas as pd
import torch
import importlib
import logging
from pathlib import Path
from sklearn.model_selection import GroupKFold, GroupShuffleSplit

# Pycox and PyTorch tuples for survival analysis
import torchtuples as tt
import pycox
from pycox.preprocessing.label_transforms import LabTransDiscreteTime
from pycox.models import CoxPH, DeepHit
from pycox.evaluation import EvalSurv

# Ray for hyperparameter tuning and distributed processing
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.search import ConcurrencyLimiter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.air import session
import ray.cloudpickle as pickle

# Custom modules for data handling, balancing, training, evaluation, and model architectures
import dataloader2
import databalancer2
import datatrainer2
import modeleval
import netweaver2

# Reload custom modules to ensure latest changes are available
importlib.reload(dataloader2)
importlib.reload(databalancer2)
importlib.reload(datatrainer2)
importlib.reload(modeleval)
importlib.reload(netweaver2)

# Import specific functions from custom modules to keep code clean and readable
from netweaver2 import (
    lstm_net_init, DHANNWrapper, LSTMWrapper, generalized_ann_net_init
)
from dataloader2 import (
    load_and_transform_data, preprocess_data #stack_sequences, dh_dataset_loader
)
from databalancer2 import (
    define_medoid_general, df_event_focus, rebalance_data, underbalance_data_general, medoid_cluster, 
    dh_rebalance_data
)
from datatrainer2 import (
    recursive_clustering, prepare_training_data, 
    prepare_validation_data, lstm_training
)
from modeleval import (
    dh_test_model, nam_dagostino_chi2, get_baseline_hazard_at_timepoints, combined_test_model
)

import psutil
torch.cuda.empty_cache()
gc.collect()

80

### 2. Define constants, load datasets

In [2]:
# Define Constants and Load Datasets
RANDOM_SEED = 12345
N_SPLIT = 2
FEATURE_COLS = ['gender', 'dm', 'ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
DURATION_COL = 'date_from_sub_60'
EVENT_COL = 'endpoint'
CLUSTER_COL = 'key'
TIME_GRID = np.array([i * 365 for i in range(6)])

# Define Feature Groups
CAT_FEATURES = ['gender', 'dm', 'ht', 'sprint']
LOG_FEATURES = ['a1c', 'po4', 'UACR_mg_g', 'Cr']
STANDARD_FEATURES = ['age', 'alb', 'ca', 'hb', 'hco3']
PASSTHROUGH_FEATURES = ['key', 'date_from_sub_60', 'endpoint']

# Load and Transform Data
BASE_FILENAME = '/mnt/d/pydatascience/g3_regress/data/X/X_20240628'
X_train_transformed, X_test_transformed = load_and_transform_data(
    BASE_FILENAME, CAT_FEATURES, LOG_FEATURES, STANDARD_FEATURES, PASSTHROUGH_FEATURES
)

2024-11-11 01:11:14,040 - INFO - Transforming training data...
2024-11-11 01:11:28,099 - INFO - Transforming test data...


### 3. Train and save models
- model naming: {deepsurv/deephit}\_{nn}\_{resample method}_{outcome}
- for deepsurv model, only the result in time_grid will be retrieved so the result of deepsurv and deephit models are compatible

In [3]:
def create_neural_network(config, num_risk = len(X_train_transformed[EVENT_COL].unique()) - 1, num_time_bins=len(TIME_GRID)):
    """
    Function to create a neural network based on the given configuration.

    Args:
        config (dict): Configuration dictionary containing model type, network type, and hyperparameters.

    Returns:
        torch.nn.Module: Created neural network model.
    """
    gc.collect()
    torch.cuda.empty_cache()
    if config['model'] == 'deepsurv':
        num_risk = None
        num_time_bins=None
    elif config['model'] == 'deephit':
        num_risk = num_risk
        num_time_bins = num_time_bins
    # Create the Neural Network
    if config['net'] == 'ann':
        net = generalized_ann_net_init(
            input_size=len(config['features']),
            num_nodes=config["num_nodes"],
            batch_norm=config["batch_norm"],
            dropout=config["dropout"],
            output_size=1, # Default output size for DeepSurv
            num_risks = num_risk,
            num_time_bins = num_time_bins
        )
    elif config['net'] == 'lstm':
        net = lstm_net_init(
            input_size=len(config['features']),
            num_nodes=config["num_nodes"],
            batch_norm=config["batch_norm"],
            dropout=config["dropout"],
            num_risks = num_risk,
            num_time_bins = num_time_bins
        )
    else:
        raise ValueError("Unknown network type: {}".format(config['net']))

    optimizer = tt.optim.AdamWR(decoupled_weight_decay=1e-6, cycle_eta_multiplier=0.8)
    if config['model'] == 'deepsurv':
        model = CoxPH(net, optimizer)
    elif config['model'] == 'deephit':
        model = DeepHit(net, optimizer)
    model.optimizer.set_lr(config["lr"])
    
    return model

def train_neural_network(model, config, X_train, X_val, duration_col, event_col, cluster_col, callbacks, time_grid=None):
    """
    Function to train a given neural network using the provided datasets.

    Args:
        net (torch.nn.Module): Neural network to be trained.
        config (dict): Configuration dictionary containing model hyperparameters.
        X_train (pd.DataFrame): Training dataset with features.
        X_val (pd.DataFrame): Validation dataset with features.
        duration_col (str): Column representing event durations.
        event_col (str): Column representing event occurrences.
        cluster_col (str): Column for grouping during cross-validation.
        callbacks (list): List of callbacks for training.
        time_grid (np.array, optional): Time grid for evaluation if required. Defaults to None.

    Returns:
        model: Trained PyCox model.
        logs: Training logs.
    """
    gc.collect()
    torch.cuda.empty_cache()
    # Train the model
    if config['model'] == 'deepsurv':
        print('Initiate training of deepsurv neural network')
        X_val = df_event_focus(X_val, event_col, config['endpoint'])
        X_val_processed, y_val = preprocess_data(X_val, config['features'], duration_col, event_col)
        val_data = (X_val_processed, y_val)
        if config['net'] == 'ann':
            print('model structure: ANN')
            if config['balance_method'] == 'clustering':
                print('data balancing method: clustering')
                model, logs = recursive_clustering(model, X_train, duration_col, event_col, config, val_data, callbacks, max_repeats=30)
            elif config['balance_method'] == 'enn':
                print('data balancing method: smoteenn')
                X_train = rebalance_data(X_train, event_col, config['endpoint'], CAT_FEATURES, config, RANDOM_SEED, method='ENN')
                X_train, y_train = preprocess_data(X_train, config['features'], duration_col, event_col)
                logs = model.fit(X_train, y_train, config['batch_size'], int(config['max_epochs']), callbacks, verbose=True, val_data=val_data, num_workers=10)
            elif config['balance_method'] == 'tomek':
                print('data balancing method: smotetomek')
                X_train = rebalance_data(X_train, event_col, config['endpoint'], CAT_FEATURES, config, RANDOM_SEED, method='Tomek')
                X_train, y_train = preprocess_data(X_train, config['features'], duration_col, event_col)
                logs = model.fit(X_train, y_train, config['batch_size'], int(config['max_epochs']), callbacks, verbose=True, val_data=val_data, num_workers=10)
        elif config['net'] == 'lstm':
            print('model structure: LSTM')
            if config['balance_method'] == 'clustering':
                print('data balancing method: clustering')
                model, logs = lstm_training(model, X_train, X_val, duration_col, event_col, cluster_col, config, callbacks, time_grid)
            elif config['balance_method'] == 'NearMiss':
                print('data balancing method: NearMiss')
                model, logs = lstm_training(model, X_train, X_val, duration_col, event_col, cluster_col, config, callbacks, time_grid)
    elif config['model'] == 'deephit':
        print('Initiate training of deephit neural network')
        X_val_processed, y_val = preprocess_data(X_val, config['features'], duration_col, event_col, TIME_GRID, discretize=True)
        val_data = (X_val_processed, y_val)
        if config['net'] == 'ann':
            print('model structure: ANN')
            if config['balance_method'] == 'clustering':
                print('data balancing method: clustering')
                model, logs = recursive_clustering(model, X_train, duration_col, event_col, config, val_data, callbacks, max_repeats=30, time_grid=TIME_GRID)
            elif config['balance_method'] == 'NearMiss':
                print('data balancing method: NearMiss')
                X_train = underbalance_data_general(X_train, EVENT_COL, CLUSTER_COL, config, version=config['version'])
                X_train, y_train = preprocess_data(X_train, config['features'], duration_col, event_col, TIME_GRID, discretize=True)
                logs = model.fit(X_train, y_train, config['batch_size'], int(config['max_epochs']), callbacks, verbose=True, val_data=val_data)
        elif config['net'] == 'lstm':
            print('model structure: LSTM')
            if config['balance_method'] == 'clustering':
                print('data balancing method: clustering')
                model, logs = lstm_training(model, X_train, X_val, duration_col, event_col, cluster_col, config, callbacks, time_grid)
            elif config['balance_method'] == 'NearMiss':
                print('data balancing method: NearMiss')
                model, logs = lstm_training(model, X_train, X_val, duration_col, event_col, cluster_col, config, callbacks, time_grid)        

    # Free memory after training
    gc.collect()
    torch.cuda.empty_cache()

    return model, logs

def save_model(params, model, model_path, baseline_hazard_path):
    """
    Save model weights and baseline hazard data.

    Parameters:
    - model: The trained model to save.
    - model_path: Path to save the model weights (.pt file).
    - baseline_hazard_path: Path to save the baseline hazards (.pkl file).
    """
    # Compute baseline hazards and save
    if params['model'] == 'deepsurv':
        baseline_hazard = model.compute_baseline_hazards()
        baseline_hazard.to_pickle(baseline_hazard_path)
    
    # Save model weights
    model.save_model_weights(model_path)
    print(f"Model and baseline hazards saved to {model_path} and {baseline_hazard_path}.")

def training_wrapper(df, config, spliter, model_path, hazard_path, feature_col=FEATURE_COLS, duration_col=DURATION_COL, event_col=EVENT_COL, cluster_col=CLUSTER_COL, time_grid=TIME_GRID):
    """
    Train and save a survival analysis model with grouped cross-validation splits.

    This function performs training on grouped cross-validation splits of the input DataFrame and saves each trained model
    along with its baseline hazards. Memory management is handled to ensure efficient GPU usage.

    Parameters:
    - df (pd.DataFrame): DataFrame containing training data.
    - config (dict): Configuration dictionary for initializing the neural network.
    - spliter (object): Splitter object (e.g., GroupShuffleSplit or StratifiedKFold) used for creating train-validation splits.
    - model_path (str): File path to save the trained model weights (.pt file).
    - hazard_path (str): File path to save the baseline hazards (.pkl file).
    - feature_col (list): List of feature column names in `df` used for model training.
    - duration_col (str): Name of the column representing duration/time-to-event.
    - event_col (str): Name of the column representing the event indicator (0 = censored, 1 = event).
    - cluster_col (str): Name of the column used for grouping (clusters for cross-validation).
    - time_grid (list): List or array defining the time grid for training.

    Returns:
    - None: Saves the model weights and baseline hazard data for each cross-validation split.
    """
    for train_idx, val_idx in spliter.split(X=df[feature_col], y=df[event_col], groups=df[cluster_col]):
        # Clear GPU memory for each split
        gc.collect()
        torch.cuda.empty_cache()
        
        # Define early stopping callback
        callbacks = [tt.cb.EarlyStopping()]
        
        # Create training and validation sets
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]
        
        # Initialize and train the model
        model = create_neural_network(config)
        model, logs = train_neural_network(
            model, config,
            X_train=train_df, X_val=val_df,
            duration_col=duration_col, event_col=event_col,
            cluster_col=cluster_col, callbacks=callbacks, time_grid=time_grid
        )
        
        # Save the trained model and its baseline hazards
        save_model(config, model, model_path, hazard_path)
        
        # Free memory for the next iteration
        del model, logs
        gc.collect()
        torch.cuda.empty_cache()

    print("Training and saving completed for all cross-validation splits.")

    print("All models have been trained and saved successfully.")

#### 3.1 deepsurv_ann_clustering_1
- features: ['gender', 'dm', 'ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling strategy: 0.05
- 2 hidden layers with 8 and 4 nodes
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.1144793446270997
- learning rate: 0.1
- max epochs: 9
- batch size: 512

In [4]:
gc.collect()
torch.cuda.empty_cache()

deepsurv_ann_clustering_1_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'clustering',
    'features': ['gender', 'dm', 'ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 1,
    'num_nodes': [8, 4],
    'batch_norm': False,
    'dropout': 0.1144793446270997,
    'lr': 0.1,
    'max_epochs': 9,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.2 deepsurv_ann_smoteenn_1
- features: ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3']
- sampling strategy: 0.3
- 4 hidden layers with 64, 32, 16 and 8 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.09555033386059111
- learning rate: 0.1
- max epochs: 16
- batch size: 512

In [5]:
deepsurv_ann_smoteenn_1_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'enn',
    'features': ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3'],
    'endpoint': 1,
    'num_nodes': [64, 32, 16, 8],
    'batch_norm': True,
    'dropout': 0.09555033386059111,
    'lr': 0.1,
    'max_epochs': 16,
    'batch_size': 512,
    'sampling_strategy': 0.3,
    'seq_length': 1,
}

#### 3.3 deepsurv_ann_smotetomek_1
- features:  ['gender', 'dm', 'ht', 'sprint', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3']
- sampling strategy: 0.2
- 3 hidden layers with 32, 16 and 8 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.23872991564684112
- learning rate: 0.1
- max epochs: 14
- batch size: 512

In [6]:
deepsurv_ann_smotetomek_1_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'tomek',
    'features': ['gender', 'dm', 'ht', 'sprint', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3'],
    'endpoint': 1,
    'num_nodes': [32, 16, 8],
    'batch_norm': True,
    'dropout': 0.23872991564684112,
    'lr': 0.1,
    'max_epochs': 14,
    'batch_size': 512,
    'sampling_strategy': 0.2,
    'seq_length': 1,
}

#### 3.4 deepsurv_ann_clustering_2
- features: ["gender", "a1c", "po4", "UACR_mg_g", "Cr"]
- sampling_strategy: 0.05
- 3 hidden layers with 32, 16, 8 nodes
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.3058921011568742
- learning rate: 0.1
- max epochs: 14
- batch size: 512

In [7]:
deepsurv_ann_clustering_2_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'clustering',
    'features': ["gender", "a1c", "po4", "UACR_mg_g", "Cr"],
    'endpoint': 2,
    'num_nodes': [32, 16, 8],
    'batch_norm': False,
    'dropout': 0.3058921011568742,
    'lr': 0.1,
    'max_epochs': 14,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.5 deepsurv_ann_smoteenn_2
- features: ["gender", "dm", "ht", "sprint", "a1c", "po4", "UACR_mg_g", "Cr", "age", "alb", "ca", "hb", "hco3"]
- sampling_strategy: 0.1, 
- 2 hidden layers with 8, 4 nodes
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.38878203553667456
- learning rate: 0.01
- max epochs: 10
- batch size: 512

In [8]:
deepsurv_ann_smoteenn_2_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'enn',
    'features': ["gender", "dm", "ht", "sprint", "a1c", "po4", "UACR_mg_g", "Cr", "age", "alb", "ca", "hb", "hco3"],
    'endpoint': 2,
    'num_nodes': [8, 4],
    'batch_norm': False,
    'dropout': 0.38878203553667456,
    'lr': 0.01,
    'max_epochs': 10,
    'batch_size': 512,
    'sampling_strategy': 0.1,
    'seq_length': 1,
}

#### 3.6 deepsurv_ann_smotetomek_2
- features: ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling_strategy: 0.05
- 2 hidden layers with 64, 32 nodes
- batch normalization in each hidden layer 
- dropout ratio in each layer: 0.3162398297390827
- learning rate: 0.1
- max epochs: 11
- batch size: 512

In [9]:
deepsurv_ann_smotetomek_2_config = {
    'model': 'deepsurv',
    'net': 'ann',
    'balance_method': 'tomek',
    'features': ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 2,
    'num_nodes': [64, 32],
    'batch_norm': True,
    'dropout': 0.3162398297390827,
    'lr': 0.1,
    'max_epochs': 11,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.7 deepsurv_lstm_clustering_1
- features: ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr']
- sampling_strategy: 0.05
- 3 hidden layers with 8, 4, 2 nodes
- sequence length 7
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.2772567071863989
- learning rate: 0.1
- max epochs: 13
- batch size: 512

In [10]:
deepsurv_lstm_clustering_1_config = {
    'model': 'deepsurv',
    'net': 'lstm',
    'balance_method': 'clustering',
    'features': ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr'],
    'endpoint': 1,
    'num_nodes': [8, 4, 2],
    'batch_norm': False,
    'dropout': 0.2772567071863989,
    'lr': 0.1,
    'max_epochs': 13,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 7,
}

#### 3.8 deepsurv_lstm_nearmiss_1
- features: ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr']
- sampling_strategy: 0.05
- 3 hidden layers with 8, 4, 2 nodes
- seq_length': 8
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.3397308077824205
- learning rate: 0.001
- max epochs: 9
- batch size: 512

In [11]:
deepsurv_lstm_nearmiss_1_config = {
    'model': 'deepsurv',
    'net': 'lstm',
    'balance_method': 'NearMiss',
    'features': ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr'],
    'endpoint': 1,
    'num_nodes': [8, 4, 2],
    'batch_norm': False,
    'dropout': 0.3397308077824205,
    'lr': 0.001,
    'max_epochs': 9,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 8,
}

#### 3.9 deepsurv_lstm_clustering_2
- features: ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr']
- sampling_strategy: 0.05
- 3 hidden layers with 8, 4, 2 nodes
- seq_length': 8
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.3397308077824205
- learning rate: 0.001
- max epochs: 9
- batch size: 512

In [12]:
deepsurv_lstm_clustering_2_config = {
    'model': 'deepsurv',
    'net': 'lstm',
    'balance_method': 'clustering',
    'features': ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr'],
    'endpoint': 2,
    'num_nodes': [8, 4, 2],
    'batch_norm': False,
    'dropout': 0.3397308077824205,
    'lr': 0.001,
    'max_epochs': 9,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 8,
}

#### 3.10 deepsurv_lstm_nearmiss_2
- features: ['sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling_strategy: 0.05
- 2 hidden layers with 32, 16 nodes
- seq_length': 2
- no batch normalization in each hidden layer
- dropout ratio in each layer: 0.35763396978044143
- learning rate: 0.1
- max epochs: 10
- batch size: 512

In [13]:
deepsurv_lstm_nearmiss_2_config = {
    'model': 'deepsurv',
    'net': 'lstm',
    'balance_method': 'NearMiss',
    'features': ['sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 2,
    'num_nodes': [32, 16],
    'batch_norm': False,
    'dropout': 0.35763396978044143,
    'lr': 0.1,
    'max_epochs': 10,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 2,
}

#### 3.11 deephit_ann_clustering_all
- features: ['gender', 'dm', 'ht', 'sprint', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3']
- sampling strategy: 0.05
- 2 hidden layers with 64 and 32 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.26400151710698067
- learning rate: 0.1
- max epochs: 8
- batch size: 512

In [14]:
deephit_ann_clustering_all_config = {
    'model': 'deephit',
    'net': 'ann',
    'balance_method': 'clustering',
    'features': ['gender', 'dm', 'ht', 'sprint', 'po4', 'UACR_mg_g', 'Cr', 'age', 'hb', 'hco3'],
    'endpoint': 'all',
    'num_nodes': [64, 32],
    'batch_norm': True,
    'dropout': 0.26400151710698067,
    'lr': 0.1,
    'max_epochs': 8,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.12 deephit_ann_nearmiss2_all
- features: ['sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling strategy: 0.05
- 2 hidden layers with 8, 4 and 2 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.7346754269827496
- learning rate: 0.01
- max epochs: 7
- batch size: 512

In [15]:
deephit_ann_nearmiss2_all_config = {
    'model': 'deephit',
    'net': 'ann',
    'balance_method': 'NearMiss',
    'version': 2,
    'features': ['sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 'all',
    'num_nodes': [8, 4, 2],
    'batch_norm': True,
    'dropout': 0.7346754269827496,
    'lr': 0.01,
    'max_epochs': 7,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 1,
}

#### 3.13 deephit_lstm_clustering_all
- features: ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3']
- sampling strategy: 0.05
- seq_length: 6
- 3 hidden layers with 64, 32 and 16 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.46132889488306583
- learning rate: 0.1
- max epochs: 5
- batch size: 512

In [16]:
deephit_lstm_clustering_all_config = {
    'model': 'deephit',
    'net': 'lstm',
    'balance_method': 'clustering',
    'version': 2,
    'features': ['ht', 'sprint', 'a1c', 'po4', 'UACR_mg_g', 'Cr', 'age', 'alb', 'ca', 'hb', 'hco3'],
    'endpoint': 'all',
    'num_nodes': [64, 32, 16],
    'batch_norm': True,
    'dropout': 0.46132889488306583,
    'lr': 0.1,
    'max_epochs': 5,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 6,
}

#### 3.14 deephit_lstm_nearmiss1_all
- features: ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr']
- sampling strategy: 0.05
- seq_length: 9
- 3 hidden layers with 32, 16 and 8 nodes
- batch normalization in each hidden layer
- dropout ratio in each layer: 0.18001924589390816
- learning rate: 0.1
- max epochs: 9
- batch size: 512

In [17]:
deephit_lstm_nearmiss1_all_config = {
    'model': 'deephit',
    'net': 'lstm',
    'balance_method': 'NearMiss',
    'version': 1,
    'features': ['gender', 'a1c', 'po4', 'UACR_mg_g', 'Cr'],
    'endpoint': 'all',
    'num_nodes': [32, 16, 8],
    'batch_norm': True,
    'dropout': 0.18001924589390816,
    'lr': 0.1,
    'max_epochs': 9,
    'batch_size': 512,
    'sampling_strategy': 0.05,
    'seq_length': 9,
}

In [18]:
model_ls = ['deepsurv_ann_clustering_1', 'deepsurv_ann_smoteenn_1', 'deepsurv_ann_smotetomek_1',
            'deepsurv_ann_clustering_2', 'deepsurv_ann_smoteenn_2', 'deepsurv_ann_smotetomek_2',
            'deepsurv_lstm_clustering_1', 'deepsurv_lstm_nearmiss', 'deepsurv_lstm_clustering_2', 'deepsurv_lstm_nearmiss_2',
            'deephit_ann_clustering_all', 'deephit_ann_nearmiss2_all', 'deephit_lstm_clustering_all', 'deephit_lstm_nearmiss1_all']
model_path = '/mnt/d/PYDataScience/g3_regress/code/models/'

In [19]:
gss1 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
gss2 = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=RANDOM_SEED)
for train_idx_1, fin_val_idx in gss1.split(X=X_train_transformed[FEATURE_COLS], y=X_train_transformed[EVENT_COL], groups=X_train_transformed[CLUSTER_COL]):
    X_train_transformed_2, X_fin_val = X_train_transformed.iloc[train_idx_1, :], X_train_transformed.iloc[fin_val_idx, :]
    gc.collect()
    torch.cuda.empty_cache()
    for model in model_ls:
        config_var_name = model + "_config"
        model_config = globals().get(config_var_name)
        if model_config is None:
            print(f"Configuration for {config_var_name} not found.")
            continue

        model_weights_path = f'{model_path}{model}.pt'
        model_hazard_path = f'{model_path}{model}_hazard.pkl'
        
        training_wrapper(X_train_transformed_2, model_config, gss2, model_weights_path, 
                        model_hazard_path, 
                        feature_col=FEATURE_COLS, duration_col=DURATION_COL, event_col=EVENT_COL, cluster_col=CLUSTER_COL, time_grid=TIME_GRID)
        gc.collect()
        torch.cuda.empty_cache()

2024-11-11 01:11:29,990 - INFO - Event column 'endpoint' updated with focus on event value 1.
2024-11-11 01:11:29,999 - INFO - Event column 'endpoint' updated with focus on event value 1.
2024-11-11 01:11:30,015 - INFO - Performing clustering iteration 1 / 20
2024-11-11 01:11:30,015 - INFO - init
2024-11-11 01:11:30,017 - INFO - CUDA environment set up and GPU memory cleared.
2024-11-11 01:11:30,022 - INFO - Event column 'endpoint' updated with focus on event value 1.


Initiate training of deepsurv neural network
model structure: ANN
data balancing method: clustering


2024-11-11 01:11:31,098 - INFO - Defined medoid for deepsurv model with 1207 clusters.
/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/callbacks.py:607: UserWarning: This overload of add is deprecated:
	add(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add(Tensor other, *, Number alpha = 1) (Triggered internally at ../torch/csrc/utils/python_arg_parser.cpp:1581.)
  p.data = p.data.add(-weight_decay * eta, p.data)


0:	[0s / 0s],		train_loss: 4.9830,	val_loss: 7.6472
1:	[0s / 0s],		train_loss: 4.7894,	val_loss: 7.4912
2:	[0s / 0s],		train_loss: 4.7147,	val_loss: 7.4432
3:	[0s / 0s],		train_loss: 4.6912,	val_loss: 7.4037
4:	[0s / 0s],		train_loss: 4.6718,	val_loss: 7.4363
5:	[0s / 0s],		train_loss: 4.6559,	val_loss: 7.3902
6:	[0s / 0s],		train_loss: 4.6536,	val_loss: 7.3836
7:	[0s / 0s],		train_loss: 4.6475,	val_loss: 7.4139
8:	[0s / 0s],		train_loss: 4.6416,	val_loss: 6.8589


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

...

64249


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torch/nn/modules/rnn.py:917: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters(). (Triggered internally at ../aten/src/ATen/native/cudnn/RNN.cpp:1424.)
  result = _VF.lstm(input, hx, self._flat_weights, self.bias, self.num_layers,


0:	[1s / 1s],		train_loss: 0.4642,	val_loss: 0.0951
1:	[1s / 2s],		train_loss: 0.4103,	val_loss: 0.0848
2:	[1s / 4s],		train_loss: 0.3984,	val_loss: 0.0821
3:	[1s / 5s],		train_loss: 0.3965,	val_loss: 0.0723
4:	[1s / 6s],		train_loss: 0.3833,	val_loss: 0.0728
5:	[1s / 8s],		train_loss: 0.3557,	val_loss: 0.0676
6:	[3s / 12s],		train_loss: 0.3368,	val_loss: 0.0642
7:	[1s / 13s],		train_loss: 0.3466,	val_loss: 0.0571
8:	[1s / 15s],		train_loss: 0.3259,	val_loss: 0.0536


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards saved to /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_nearmiss1_all.pt and /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_nearmiss1_all_hazard.pkl.
Training and saving completed for all cross-validation splits.
All models have been trained and saved successfully.


### 4. Load models amd hazards

In [20]:
def load_model(model, model_config, model_path, baseline_hazard_path):
    """
    Load model weights and baseline hazard data.

    Parameters:
    - create_model_func: Function to create the model architecture (e.g., create_neural_network).
    - model_path: Path to load the model weights (.pt file).
    - baseline_hazard_path: Path to load the baseline hazards (.pkl file).

    Returns:
    - model: The loaded model with weights and baseline hazards.
    """
    
    # Load model weights
    model.load_model_weights(model_path)
    
    # Load baseline hazards and assign to model
    if model_config['model'] == 'deepsurv':
        baseline_hazard = pd.read_pickle(baseline_hazard_path)
        model.baseline_hazards_ = baseline_hazard
        model.baseline_cumulative_hazards_ = baseline_hazard.cumsum()
    
    print(f"Model and baseline hazards loaded from {model_path} and {baseline_hazard_path}.")
    return model

# model_ls = ['deepsurv_ann_clustering_1', 'deepsurv_ann_smoteenn_1', 'deepsurv_ann_smotetomek_1',
#             'deepsurv_ann_clustering_2', 'deepsurv_ann_smoteenn_2', 'deepsurv_ann_smotetomek_2',
#             'deepsurv_lstm_clustering_1', 'deepsurv_lstm_nearmiss', 'deepsurv_lstm_clustering_2', 'deepsurv_lstm_nearmiss_2',
#             'deephit_ann_clustering_all', 'deephit_ann_nearmiss2_all', 'deephit_lstm_clustering_all', 'deephit_lstm_nearmiss1_all']
# model_path = '/mnt/d/PYDataScience/g3_regress/code/models/'

In [24]:
# Dictionary to store loaded models
loaded_models = {}

for model_name in model_ls:
    # Retrieve configuration by dynamically constructing the variable name
    config_var_name = model_name + "_config"
    model_config = globals().get(config_var_name)
    
    if model_config is None:
        print(f"Configuration for {config_var_name} not found.")
        continue

    model_weights_path = f'{model_path}{model_name}.pt'
    model_hazard_path = f'{model_path}{model_name}_hazard.pkl'
    
    # Define the model creation function as a lambda to pass the config
    create_model_func = lambda: create_neural_network(
        config=model_config,
        num_risk=len(X_train_transformed[EVENT_COL].unique()) - 1,
        num_time_bins=len(TIME_GRID)
    )
    model = create_model_func()
    
    # Load the model and store it in the dictionary
    loaded_models[model_name] = load_model(model, model_config, model_weights_path, model_hazard_path)
    print(f'Loaded model {model_name}')

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_clustering_1.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_clustering_1_hazard.pkl.
Loaded model deepsurv_ann_clustering_1
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smoteenn_1.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smoteenn_1_hazard.pkl.
Loaded model deepsurv_ann_smoteenn_1


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smotetomek_1.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smotetomek_1_hazard.pkl.
Loaded model deepsurv_ann_smotetomek_1
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_clustering_2.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_clustering_2_hazard.pkl.
Loaded model deepsurv_ann_clustering_2


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smoteenn_2.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smoteenn_2_hazard.pkl.
Loaded model deepsurv_ann_smoteenn_2
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smotetomek_2.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_ann_smotetomek_2_hazard.pkl.
Loaded model deepsurv_ann_smotetomek_2


...

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_clustering_1.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_clustering_1_hazard.pkl.
Loaded model deepsurv_lstm_clustering_1
Configuration for deepsurv_lstm_nearmiss_config not found.
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_clustering_2.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_clustering_2_hazard.pkl.
Loaded model deepsurv_lstm_clustering_2


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_nearmiss_2.pt and /mnt/d/PYDataScience/g3_regress/code/models/deepsurv_lstm_nearmiss_2_hazard.pkl.
Loaded model deepsurv_lstm_nearmiss_2
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deephit_ann_clustering_all.pt and /mnt/d/PYDataScience/g3_regress/code/models/deephit_ann_clustering_all_hazard.pkl.
Loaded model deephit_ann_clustering_all


/home/goma/miniconda3/envs/ai_dev/lib/python3.11/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.net.load_state_dict(torch.load(pat

Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deephit_ann_nearmiss2_all.pt and /mnt/d/PYDataScience/g3_regress/code/models/deephit_ann_nearmiss2_all_hazard.pkl.
Loaded model deephit_ann_nearmiss2_all
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_clustering_all.pt and /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_clustering_all_hazard.pkl.
Loaded model deephit_lstm_clustering_all
Model and baseline hazards loaded from /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_nearmiss1_all.pt and /mnt/d/PYDataScience/g3_regress/code/models/deephit_lstm_nearmiss1_all_hazard.pkl.
Loaded model deephit_lstm_nearmiss1_all


### 5. Prepare test dataset and make prediction on the 

In [21]:
def predict_neural_network(model, config, X_test, duration_col, event_col, cluster_col, time_grid=None):
    """
    Function to train a given neural network using the provided datasets.

    Args:
        net (torch.nn.Module): Neural network to be trained.
        config (dict): Configuration dictionary containing model hyperparameters.
        X_train (pd.DataFrame): Training dataset with features.
        X_val (pd.DataFrame): Validation dataset with features.
        duration_col (str): Column representing event durations.
        event_col (str): Column representing event occurrences.
        cluster_col (str): Column for grouping during cross-validation.
        callbacks (list): List of callbacks for training.
        time_grid (np.array, optional): Time grid for evaluation if required. Defaults to None.

    Returns:
        model: Trained PyCox model.
        logs: Training logs.
    """
    gc.collect()
    torch.cuda.empty_cache()
    # Train the model
    if config['model'] == 'deepsurv':
        print('Initiate testing of deepsurv neural network')
        X_test = df_event_focus(X_test, event_col, config['endpoint'])
        if config['net'] == 'ann':
            print('model structure: ANN')
            X_test_processed, y_test = preprocess_data(X_test, config['features'], duration_col, event_col)
            surv = model.predict_surv_df(X_test_processed, batch_size=512)
        elif config['net'] == 'lstm':
            print('model structure: LSTM')
            X_test_processed, y_test = prepare_validation_data(X_test, config['features'], duration_col, event_col, config, cluster_col, config['model'], time_grid)
            X_test_tensor = torch.tensor(X_test_processed, dtype=torch.float32)
            y_test_tensor = (torch.tensor(y_test[0], dtype=torch.float32), torch.tensor(y_test[1], dtype=torch.float32))
            surv = model.predict_surv_df(X_test_tensor, batch_size=512)
    elif config['model'] == 'deephit':
        print('Initiate testing of deephit neural network')
        if config['net'] == 'ann':
            print('model structure: ANN')
            X_test_processed, y_test = preprocess_data(X_test, config['features'], duration_col, event_col, time_grid, discretize=True)
            surv = model.predict_cif(X_test_processed, batch_size=512)
            print('prediction complete, please note that prediction of deephit models are CIF.')
        elif config['net'] == 'lstm':
            print('model structure: LSTM')
            X_test_processed, y_test = prepare_validation_data(X_test, config['features'], duration_col, event_col, config, cluster_col, config['model'], time_grid)
            surv = model.predict_cif(X_test_processed, batch_size=512)
            print('prediction complete, please note that prediction of deephit models are CIF.')

    # Free memory after training
    gc.collect()
    torch.cuda.empty_cache()

    return surv, y_test

In [22]:

def align_to_time_grid(surv, time_grid):
    """
    Align the survival DataFrame to the closest indices of the time grid.

    Parameters:
        surv (pd.DataFrame): Survival probabilities DataFrame.
        time_grid (np.array): Array of target time points to align.

    Returns:
        aligned_surv (pd.DataFrame): Aligned survival probabilities.
    """
    # Convert the DataFrame's index to a NumPy array for fast computation
    surv_times = np.array(surv.index)
    
    # Find the closest time in the survival DataFrame for each time in the grid
    closest_indices = [np.argmin(np.abs(surv_times - t)) for t in time_grid]
    
    # Extract the rows corresponding to the closest times
    aligned_surv = surv.iloc[closest_indices].copy()
    
    # Reindex the DataFrame to match the time grid
    aligned_surv.index = range(len(time_grid))  # Standardize indices to 0, 1, 2, ...
    return aligned_surv

In [33]:
gc.collect()
surv, y_test = predict_neural_network(model=loaded_models['deepsurv_ann_smoteenn_1'], config=deepsurv_ann_smoteenn_1_config, X_test=X_train_transformed_2, duration_col=DURATION_COL,
                    event_col=EVENT_COL, cluster_col=CLUSTER_COL, time_grid=TIME_GRID)
surv = align_to_time_grid(surv, TIME_GRID)
gc.collect()

2024-11-11 00:59:25,848 - INFO - Event column 'endpoint' updated with focus on event value 1.


Initiate testing of deepsurv neural network
model structure: ANN


KeyboardInterrupt: 

In [43]:
gc.collect()
surv, y_test = predict_neural_network(model=loaded_models['deephit_ann_nearmiss2_all'], config=deephit_ann_nearmiss2_all_config, X_test=X_train_transformed_2, duration_col=DURATION_COL,
                    event_col=EVENT_COL, cluster_col=CLUSTER_COL, time_grid=TIME_GRID)
surv1 = pd.DataFrame(surv[0], index=loaded_models['deephit_ann_nearmiss2_all'].duration_index)
ev = EvalSurv(1- surv1, y_test[0], y_test[1] == 1, censor_surv='km')
display(ev.concordance_td())
gc.collect()

Initiate testing of deephit neural network
model structure: ANN
prediction complete, please note that prediction of deephit models are CIF.


0.7593451986384061

0

In [64]:
# model_ls = ['deepsurv_ann_clustering_1', 'deepsurv_ann_smoteenn_1', 'deepsurv_ann_smotetomek_1',
#             'deepsurv_ann_clustering_2', 'deepsurv_ann_smoteenn_2', 'deepsurv_ann_smotetomek_2',
#             'deepsurv_lstm_clustering_1', 'deepsurv_lstm_nearmiss', 'deepsurv_lstm_clustering_2', 'deepsurv_lstm_nearmiss_2',
            # 'deephit_ann_clustering_all', 'deephit_ann_nearmiss2_all', 'deephit_lstm_clustering_all', 'deephit_lstm_nearmiss1_all'
            # ]
gc.collect()
torch.cuda.empty_cache()
model_predictions = {}
for model_name in model_ls:
    # Retrieve configuration by dynamically constructing the variable name
    config_var_name = model_name + "_config"
    model_config = globals().get(config_var_name)
    if model_config is None:
        print(f"Configuration for {config_var_name} not found.")
        continue
    try:
        print(f"Initiating prediction for model: {model_name}")
        
        # Retrieve the loaded model
        model = loaded_models.get(model_name)
        if model is None:
            print(f"Model {model_name} is not loaded.")
            continue
        
        # Predict using the loaded model and configuration
        surv, y_test = predict_neural_network(
            model=model,
            config=model_config,
            X_test=X_train_transformed_2,
            duration_col=DURATION_COL,
            event_col=EVENT_COL,
            cluster_col=CLUSTER_COL,
            time_grid=TIME_GRID
        )
        
        if model_config['model'] == 'deepsurv':
            surv = align_to_time_grid(surv, TIME_GRID).values  # 2D array
            
            # Structure key dynamically
            key = f"deepsurv_{model_config['net']}_{model_config['balance_method']}"
            
            # Initialize if not exists
            if key not in model_predictions:
                model_predictions[key] = np.zeros((2, *surv.shape), dtype=np.float32)
            
            # Store predictions for the corresponding endpoint
            if model_config['endpoint'] == 1:
                model_predictions[key][0] = 1 - surv
            elif model_config['endpoint'] == 2:
                model_predictions[key][1] = 1 - surv
        elif model_config['model'] == 'deephit':
            surv = np.array(surv)  # Convert to numpy array
            # Structure key dynamically
            key = f"deephit_{model_config['net']}_{model_config['balance_method']}"
            model_predictions[key] = surv
        
        print(f"Prediction completed for {model_name}.")
    
    except Exception as e:
        print(f"Error during prediction for {model_name}: {e}")

2024-11-11 00:13:58,107 - INFO - Event column 'endpoint' updated with focus on event value 1.


Initiating prediction for model: deepsurv_ann_clustering_1
Initiate testing of deepsurv neural network
model structure: ANN


2024-11-11 00:14:01,915 - INFO - Event column 'endpoint' updated with focus on event value 1.


Prediction completed for deepsurv_ann_clustering_1.
Initiating prediction for model: deepsurv_ann_smoteenn_1
Initiate testing of deepsurv neural network
model structure: ANN


: 

In [62]:
display(model_predictions['deepsurv_lstm_clustering'][0].shape)
display(model_predictions['deephit_lstm_clustering'][0].shape)

(6, 40513)

(6, 316242)